$$ \mathbf{E}(\mathbf{r}) = - k Z_m \int_{S} \Big( 1 + \frac{1}{k^2}\mathbf{\nabla}\mathbf{\nabla} \Big) G(\mathbf{r}, \mathbf{r_q}) \cdot \mathbf{J}(\mathbf{r_s}) d s_q + \int_{S} \mathbf{M}(\mathbf{r_q}) \times \mathbf{\nabla} G(\mathbf{r}, \mathbf{r_q}) d s_q $$

$$ \mathbf{H}(\mathbf{r}) = - k Z_m \int_{S} \Big( 1 + \frac{1}{k^2}\mathbf{\nabla}\mathbf{\nabla} \Big) G(\mathbf{r}, \mathbf{r_q}) \cdot \mathbf{M}(\mathbf{r_s}) d s_q + \int_{S} \mathbf{J}(\mathbf{r_q}) \times \mathbf{\nabla} G(\mathbf{r}, \mathbf{r_q}) d s_q $$


In GRASP field units of $\sqrt{\mathrm{Watts}}$ for all fields, the field at a point $\mathbf{r}$ due to induced currents on a scatterer are given by:

$$ \mathbf{E}(\mathbf{r}) = \frac{k^2}{4\pi} \int_{S} e^{-ikR} \Bigg[ Z_m \Bigg[ \mathbf{J} (\mathbf{r_s}) \Bigg(\frac{-i}{k R} - \frac{1}{k^2 R^2} + \frac{i}{k^3 R^3} \Bigg)  + \Big(\mathbf{J} (\mathbf{r_s})\cdot\mathbf{\hat{R}}\Big) \mathbf{\hat{R}} \Bigg( \frac{i}{kR} + \frac{3}{k^2 R^2} - \frac{3 i}{k^3 R^3} \Bigg) \Bigg] $$
$$ - \mathbf{M}(\mathbf{r}) \times \mathbf{\hat{R}} \Bigg( \frac{1}{k^2 R^2} + \frac{i}{kR}\Bigg) \Bigg] ds $$

$$ \mathbf{H}(\mathbf{r}) = \frac{k^2}{4\pi} \int_{S} e^{-ikR} \Bigg[ \frac{1}{Z_m} \Bigg[ \mathbf{M} (\mathbf{r_s}) \Bigg(\frac{-i}{k R} - \frac{1}{k^2 R^2} + \frac{i}{k^3 R^3} \Bigg)  + \Big(\mathbf{M} (\mathbf{r_s})\cdot\mathbf{\hat{R}}\Big) \mathbf{\hat{R}} \Bigg( \frac{i}{kR} + \frac{3}{k^2 R^2} - \frac{3 i}{k^3 R^3} \Bigg) \Bigg] $$
$$ + \mathbf{J}(\mathbf{r}) \times \mathbf{\hat{R}} \Bigg( \frac{1}{k^2 R^2} + \frac{i}{kR}\Bigg) \Bigg] ds $$